In [ ]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN
import logging
import torch
import torch.nn as nn
from qx_noface_no2d_video_swin_transformer01 import SwinTransformer3D
import gc
from einops import rearrange
from tqdm import tqdm
batchsz = 16
start_epoch = 0
test_flage=False
model_name ="model"
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./LOG/personalityLog.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)
USE_CUDA = torch.cuda.is_available()

In [ ]:
logger.debug('============={}+Video swin transformerTrain Strat!============='.format(model_name))
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

In [ ]:
def select_frame_data(datas,frame_num):
    data_set=[]
    begin_num=0
    x=128/frame_num
    indexes=[None] * frame_num
    for i in range(frame_num):
        indexes[i]=int(begin_num)
        begin_num+=x
    for data in datas:
        video_frames = [None] * 2
        video_frames[0]=[]
        video_frames[1]=[]
        for index in indexes:
            video_frames[0].append(data[0][index])
        video_frames[1].append(data[1])
        data_set.append(video_frames)
        del video_frames
        del data 
        gc.collect()
    del datas
    gc.collect()
    return data_set
print(1)

In [ ]:
def getdata(pathname,datatype):
    if datatype=="train":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            final_data_set.extend(pickle.load(output_file))
        return final_data_set
    elif datatype=="valid":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            valid_data_set=[]
            final_data_set.extend(pickle.load(output_file))
            del valid_data_set
        return final_data_set
print(1)

In [ ]:
def datasize(type,batchsz):
    if type=='train':
        i=5978#5967
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
    else:
        i=1992#1986
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
print(1)

In [ ]:

save_model_file_path = './save_swintransformer_folder/{}_{}.{}'
mtcnn= MTCNN(image_size=224, margin=0, min_face_size= 20, thresholds=[0.6, 0.7, 0.7], post_process=True)
print(1)

In [ ]:
validation_set_data=getdata("/jf-training-home/datasets_rw/issrlab-dataset2/15Frames/FaceandAudio/FaceAndAudiovalid_set.dat","valid")
validation_set_data=validation_set_data[:datasize("valid",batchsz)]
print(1)

In [ ]:
train_set_data=getdata("/jf-training-home/datasets_rw/issrlab-dataset2/15Frames/FaceandAudio/FaceAndAudiotrain_set.dat","train")
train_set_data=train_set_data[:datasize("train",batchsz)]
print(1)

In [ ]:
def imagecrop(imagedata,batchsz):
    output=[]
    for j in range(batchsz):#비디오 데이터 batchsz(9) 개 있다
        firstoutput=[]
        for i in range(len(imagedata[j])):#imagedata에 15개의 프레임이 있다
            temp = imagedata[j][i].unfold(1,112,112).unfold(2,112,112)
            temp = rearrange(temp, 'c h1 w1 h w -> (h1 w1) c h w')
            firstoutput.append(temp)
        firstoutput = torch.stack(firstoutput,0)
        output.append(firstoutput)
    output = torch.stack(output,0)
    return output
print(1)

In [ ]:
def reshape_to_expected_input(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    x0_list = []
    x1_list = []
    x2_list = []
    for i in range(0, len(dataset)):
        x0_list.append(dataset[i][0])
        x1_list.append(dataset[i][1])
        x2_list.append(dataset[i][2])
        print(len(x0_list))
        print(len(x1_list))
        print(len(x2_list))
    return (np.stack(x0_list), np.stack(x1_list), np.stack(x2_list))
print(1)

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])
print(1)

In [ ]:
class ChalearnDataset(Dataset):
    def __init__(self,audiodata,imagedata,tagdata,transform=None):
        self.audiodata=audiodata
        self.imagedata=imagedata
        self.tagdata=tagdata
        self.transform = transform  # 표준화 여부
    def __len__(self):
        return len(self.imagedata)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()#텐서의 경우 목록으로 돌아가기
        audiodata = self.audiodata[idx]
        audiodata = torch.FloatTensor(audiodata)
        image_data=self.imagedata[idx]
        imagedata1=torch.FloatTensor(image_data)
        big_five_sorces=self.tagdata[idx]
        big_five_sorces = torch.FloatTensor(big_five_sorces)
        return audiodata,imagedata1,big_five_sorces
print(1)

In [ ]:
num_workerssz = 20
lr = 3e-05
epochs = 120
print(1)

In [ ]:
train_set_data = ChalearnDataset(audiodata = train_set_data[0], imagedata=train_set_data[1],tagdata=train_set_data[2],transform=transform)
val_set_data = ChalearnDataset(audiodata = validation_set_data[0], imagedata=validation_set_data[1],tagdata=validation_set_data[2],transform=transform)
train_dataloader = DataLoader(dataset=train_set_data, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
val_dataloader = DataLoader(dataset=val_set_data, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
max_value=0
print(1)

In [ ]:
model=SwinTransformer3D()#num_frames=frame_nums,batchsz = batchsz
model.to(device)
criterion = torch.nn.L1Loss().to(device)  # 손실함수
optimizer = torch.optim.AdamW(model.parameters(), lr=lr) # 옵티마이저

if test_flage==True:
    checkpoint=torch.load(save_model_file_path.format('model',start_epoch,'pth'), map_location=device)
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])
train_avg_loss0=[]
val_avg_loss0=[]
print(1)

In [ ]:
# from torchinfo import summary

# summary(model, input_size = (8,15,224,224,3), col_names = ['input_size','output_size','num_params'])

In [ ]:
with torch.cuda.device(0):
    for i in range(start_epoch, epochs):
        train_avg_loss = 0
        val_avg_loss = 0
        
        for audiodata, image_data, big_five_data in tqdm(train_dataloader):
            big_five_data=big_five_data.permute(0,2,1)
            big_five_data=big_five_data.squeeze()
            audiodata = audiodata.to(device)
            image_data = image_data.to(device)
            big_five_data = big_five_data.to(device)
            optimizer.zero_grad()  # 기울기가 0이 됩니다.
            hypothesis = model(image_data,audiodata)  # 모델의 예측 결과를 저장합니다.
            loss = criterion(hypothesis, big_five_data)  # 예측된 결과와 실제 태그 사이의 손실 값을 저장합니다.
            loss.backward()  # 역방향 전파입니다.
            optimizer.step()  # 매개 변수를 업데이트합니다
            train_avg_loss += loss  # 훈련 손실의 평균치입니다
        train_avg_loss=train_avg_loss/len(train_dataloader)
        
        with torch.no_grad():#validate
            for audiodata, image_data, big_five_data in tqdm(val_dataloader):
                big_five_data=big_five_data.permute(0,2,1)
                big_five_data=big_five_data.squeeze()
                audiodata = audiodata.to(device)
                image_data = image_data.to(device)
                big_five_data = big_five_data.to(device)
                hypothesis = model(image_data, audiodata)
                val_loss = criterion(hypothesis, big_five_data)
                val_avg_loss += val_loss
            val_avg_loss=val_avg_loss/len(val_dataloader)

        torch.cuda.empty_cache()
        max_value=val_avg_loss
        start_epoch+=1
        if (i + 1) % 5 == 0:
            torch.save({
                    'epoch': i+1,
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'loss': val_avg_loss,
                }, save_model_file_path.format('model',start_epoch,'pth'))
        logger.debug('Epoch: {} , 1-MAE: {:.4f} , 1-train_avg_loss:{:.4f} , 1-val_avg_loss:{:.4f} , MAX Value:{:.4f} , MAX Value Epoch:{}'.format(i+1, 1-val_avg_loss, 1-train_avg_loss, 1-val_avg_loss, max_value, start_epoch))
    